In [1]:
from utils.tf import *
from utils.db import *
from utils.gd import * 
from utils.clt_repetidos import *
from datetime import datetime
import cx_Oracle
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from utils.clt_repetidos import * 
from ydata_profiling import ProfileReport
import dask.dataframe as dd
import gc
import matplotlib.pyplot as plt

# Get the data and create the output directory (Extraction)

In [12]:
path_dir_data = "/home/ale1726/proyects/datalake/clientes/data/productos/SOI/data"

In [13]:
# Obtenemos la fecha actual para crear un directorio
# donde se guardaran los archivos de salida
date_now = datetime.now().strftime("%d_%m_%Y") 
path_dir_today =  os.path.join(path_dir_data, date_now)

# Creamos el directorio si no existe
os.makedirs(path_dir_today, exist_ok=True)

In [14]:
query_productos_soi = """ 
WITH CLIENTES_ACTIVOS AS (
   SELECT  BA_BCO_FFON, BACVE_BANC, BANOM_BAN
   FROM SOI.SOI_BANCO sb
   WHERE BA_ACTIVO = 'S'
), CLT_LINEA AS (
	SELECT CA.*, SBL.BLNO_LINEA
	FROM CLIENTES_ACTIVOS CA
	LEFT JOIN SOI.SOI_BANCO_LIN SBL ON CA.BACVE_BANC = SBL.BLCVE_BANC
), LINEA_CT1 AS (
	SELECT SL.*, SB.TFDESC_CONT 
	FROM SOI.SOI_LINEA sl
	LEFT JOIN SOI.SOI_TIP_CREDI SB ON SL.LITIP_CONT = SB.TFTIP_CONT
), CLT_LINEA_DESCRIP AS (
	SELECT CL.*, SL.*
	FROM CLT_LINEA CL 
	LEFT JOIN LINEA_CT1 SL ON CL.BLNO_LINEA = SL.LINO_LINEA
) SELECT * FROM CLT_LINEA_DESCRIP
"""

In [15]:
path_dir_productos = get_table(path_dir_today, db_soi, "productos_clientes_soi", query = query_productos_soi)

# Transform the data and output the transformation

In [35]:
path_dir_output_transformation = os.path.join("/home/ale1726/proyects/datalake/clientes/data/productos/SOI/agrupados", date_now)
os.makedirs(path_dir_output_transformation, exist_ok=True)
path_dir_output_transformation

'/home/ale1726/proyects/datalake/clientes/data/productos/SOI/agrupados/22_04_2025'

In [20]:
productos_soi =  pd.read_csv(path_dir_productos, low_memory=False)

In [18]:
productos_soi.columns

Index(['BA_BCO_FFON', 'BACVE_BANC', 'BANOM_BAN', 'BLNO_LINEA', 'LINO_LINEA',
       'LITIPO_LIN', 'LIF_FIRMA', 'LIF_EFECT', 'LIMTO_LIN', 'LIMONEDA',
       'LIINT_MOR1', 'LIINT_MOR2', 'LIPLA_AMO1', 'LIPLA_AMO2', 'LIVIGENCIA',
       'LIFECHAREN', 'LIGASTLOCA', 'LIDES_GTO1', 'LIDES_GTO2', 'LINUM_SHCP',
       'LIF_SHCP', 'LIOBLISHCP', 'LICOMXLIN', 'LIACUM_DES', 'LIF_LIM_DE',
       'LITIP_CONT', 'LIPLA_AMO3', 'LIPERIOCAP', 'LIPERIOINT', 'LICVE_CORR',
       'LIISR', 'LIINT_FIN', 'LICVE_ACRE', 'LIREF_CRED', 'LIMTO_REF',
       'LISTATUS', 'LICVE_GAR', 'LICVE_DOM', 'LICTA_DOM', 'LINUEVO',
       'LIACREDIT', 'LIE_ACREDI', 'LIDIA_FER1', 'LIDIA_FER2', 'LIF_PRPACAP',
       'LIF_LIQUIDACION', 'LIPLA_AMO4', 'LI_PL_ID_ACRE', 'LI_PL_ID_GAR',
       'LI_PL_ID_DOM', 'LICVE_INTER', 'LI_PL_ID_INTER', 'LIMTO_CAP',
       'TFDESC_CONT'],
      dtype='object')

In [36]:
productos_soi["LIF_FIRMA"] = pd.to_datetime(productos_soi["LIF_FIRMA"])
productos_soi["AÑO_FFIRMA"] = productos_soi["LIF_FIRMA"].dt.year
productos_soi["MES_FFIRMA"] = productos_soi["LIF_FIRMA"].dt.month
productos_soi["MES_STR"] = productos_soi["LIF_FIRMA"].dt.month_name()  
productos_soi["LIVIGENCIA"] = pd.to_datetime(productos_soi["LIVIGENCIA"])

In [37]:
agrupado_operaciones_anuales = (
    productos_soi.groupby(['AÑO_FFIRMA'], as_index=False)
    .agg(
        num_contratos=('LINO_LINEA', 'count'), 
        monto_anuales_sum=('LIMTO_LIN', lambda x: round(x.sum(), 2)),
        monto_anuales_min=('LIMTO_LIN', 'min'), 
        monto_anuales_max=('LIMTO_LIN', 'max'), 
        monto_anuales_mean=('LIMTO_LIN', lambda x: round(x.mean(), 2)),
        monto_anuales_median=('LIMTO_LIN', 'median')
    ).sort_values(by=['AÑO_FFIRMA'], ascending=True)
)

In [38]:
agrupado_operaciones_anuales

,AÑO_FFIRMA,num_contratos,monto_anuales_sum,monto_anuales_min,monto_anuales_max,monto_anuales_mean,monto_anuales_median
0,1980.0,3,2.627304e+07,3.195948e+06,1.380851e+07,8.757678e+06,9.268573e+06
1,1981.0,3,2.258242e+07,2.402418e+06,1.500000e+07,7.527473e+06,5.179999e+06
2,1983.0,1,6.195494e+06,6.195494e+06,6.195494e+06,6.195494e+06,6.195494e+06
3,1984.0,3,4.211311e+07,8.372561e+06,1.707524e+07,1.403770e+07,1.666531e+07
4,1985.0,3,2.281332e+08,8.330000e+06,1.448000e+08,7.604439e+07,7.500318e+07
5,1986.0,2,5.698227e+07,2.654681e+07,3.043546e+07,2.849114e+07,2.849114e+07
6,1987.0,3,6.866605e+06,2.762900e+05,6.066498e+06,2.288868e+06,5.238176e+05
7,1988.0,4,3.235285e+07,2.554432e+06,1.733922e+07,8.088214e+06,6.229600e+06
8,1989.0,2,1.255954e+07,1.659545e+06,1.090000e+07,6.279772e+06,6.279772e+06
9,1990.0,2,1.238509e+08,2.385092e+07,1.000000e+08,6.192546e+07,6.192546e+07


In [39]:
agrupado_operaciones_anuales.to_csv(os.path.join(path_dir_output_transformation, "agrupado_operaciones_anuales.csv"), index=False)

In [41]:
agrupado_operaciones_anuales_mes = (
    productos_soi.groupby(['AÑO_FFIRMA','MES_FFIRMA','MES_STR'], as_index=False)
    .agg(
        num_contratos=('LINO_LINEA', 'count'), 
        monto_anuales_sum=('LIMTO_LIN', lambda x: round(x.sum(), 2)),
        monto_anuales_min=('LIMTO_LIN', 'min'), 
        monto_anuales_max=('LIMTO_LIN', 'max'), 
        monto_anuales_mean=('LIMTO_LIN', lambda x: round(x.mean(), 2)),
        monto_anuales_median=('LIMTO_LIN', 'median')
    ).sort_values(by=['AÑO_FFIRMA','MES_FFIRMA'], ascending=True)
)

In [42]:
agrupado_operaciones_anuales_mes.to_csv(os.path.join(path_dir_output_transformation, "agrupado_operaciones_anuales_mes.csv"), index=False)